<table class="tfo-notebook-buttons" align="center">
  <tr>   
    <th>
      <a target="_blank" href="https://colab.research.google.com/github.com/iamollas/hateSpeechDataV2/blob/master/setD.ipynb"> <img src="https://www.tensorflow.org/images/colab_logo_32px.png"/></a>
    </th>
    <th>
      <a target="_blank" href="https://github.com/iamollas/hateSpeechDataV2/blob/master/setD.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"/></a>
    </th>
  </tr>
  <tr>
    <th>
      <a target="_blank" href="https://colab.research.google.com/github.com/iamollas/hateSpeechDataV2/blob/master/setD.ipynb">
      Run in Google Colab</a>
    </th>
    <th>
      <a target="_blank" href="https://github.com/iamollas/hateSpeechDataV2/blob/master/setD.ipynb">
      View source on GitHub</a>
    </th>
  </tr>
</table>

Do you plan to test it on Colab? Make sure you change your settings to: Runtime->Change runtime type->Hardware accelarator->GPU

We import some necessary libraries and we load our hate speech dataset.
The preprocess script is available on the github repo.

Check code here: https://towardsdatascience.com/how-to-do-text-binary-classification-with-bert-f1348a25d905

In [1]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
#####################################################################
#                           Set D                                   #
#####################################################################
#                        BERT Tests                                 #
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
import tensorflow as tf
import nltk
import numpy as np
import time
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score
from utilities.preprocess import Preproccesor
import pandas as pd
from sklearn.model_selection import StratifiedKFold, KFold
pd.set_option('max_colwidth', 400)
nltk.download('wordnet')
nltk.download('stopwords')
X, y = Preproccesor.load_data(True)
class_names = ['noHateSpeech', 'hateSpeech']

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

We want to measure specificity and sensitivity, so we manually create those two functions for these metrics.

In [2]:
def specificity(tn, fp, fn, tp):
    tn, fp, fn, tp
    if(tn+fp) > 0:
        speci = tn/(tn+fp)
        return speci
    return 0


def sensitivity(tn, fp, fn, tp):
    tn, fp, fn, tp
    if(tp+fn) > 0:
        sensi = tp/(tp+fn)
        return sensi
    return 0

More over, for our bert experiments we will need the bert-text library in order to have more compact code

In [8]:
!pip install bert_text

  Could not find a version that satisfies the requirement bert_text (from versions: )
No matching distribution found for bert_text


Now we our testing with a 10-Fold Cross Validation process the BERT's performance. In every loop we load BERT and we perform fine tuning using the new train sets.

In [0]:
  !rm '/content/test.tsv'
  !rm '/content/train.tsv'
  !rm -R '/content/output'

In [0]:
f_results = []
n_fold = 10
folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=7)
counter = 0
for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y)):
    from bert_text import run_on_dfs
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_test = X[train_index], X[valid_index]
    y_train, y_test = y[train_index], y[valid_index]
    ids = []
    idsT = []
    for i in range(0, len(X_train)):
        ids.append(i)
    for i in range(0, len(X_test)):
        idsT.append(i)
    train = pd.DataFrame(X_train, columns=['comment_text'], index=ids)
    train['target'] = y_train
    test = pd.DataFrame(X_test, columns=['comment_text'], index=idsT)
    test['target'] = y_test
    train.to_csv('train.tsv', sep='\t', index=False, header=False)
    test.to_csv('test.tsv', sep='\t', index=False, header=True)
    myparam = {
        "DATA_COLUMN": "comment_text",
        "LABEL_COLUMN": "target",
        "MAX_SEQ_LENGTH": 100,
        "BATCH_SIZE": 16,
        "LEARNING_RATE": 2e-5,
        "NUM_TRAIN_EPOCHS": 10,
        "SAVE_SUMMARY_STEPS": 100,  # 100
        "SAVE_CHECKPOINTS_STEPS": 10000  # 10000
    }
    result, estimator = run_on_dfs(train, test, **myparam)
    f_results.append(result)
    print(len(f_results), result)
    del result, estimator
    !rm '/content/test.tsv'
    !rm '/content/train.tsv'
    !rm - R '/content/output'

Fold 0 started at Thu Sep 12 16:50:09 2019


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1 {'auc': 0.7444179, 'eval_accuracy': 0.75247526, 'f1_score': 0.70588225, 'false_negatives': 14.0, 'false_positives': 11.0, 'loss': 1.3915498, 'precision': 0.73170733, 'recall': 0.6818182, 'true_negatives': 46.0, 'true_positives': 30.0, 'global_step': 560}
Fold 1 started at Thu Sep 12 16:59:12 2019


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Finally, we compute the metrics we want and we save them to a txt file.

In [0]:
acc = 0
f1 = 0
sensi = 0
speci = 0
preci = 0
recall = 0
for i in f_results:
    tn = i['true_negatives']
    fp = i['false_positives']
    fn = i['false_negatives']
    tp = i['true_positives']
    acc = acc + i['eval_accuracy']
    f1 = f1 + i['f1_score']
    preci = preci + i['precision']
    recall = recall + i['recall']
    speci = speci + specificity(tn, fp, fn, tp)
    sensi = sensi + sensitivity(tn, fp, fn, tp)
print("{:<7} | {:<7} {:<7} {:<7} {:<7} {:<7} {:<7}".format(
    'Method', 'F1score', 'Precisi', 'Recall', 'Accurac', 'Specifi', 'Sensiti'))
print("=========================================================================")
print("{:<7} | {:<7} {:<7} {:<7} {:<7} {:<7} {:<7}".format(str("BERT")[:7], str('%.4f' % (f1/10)), str('%.4f' % (preci/10)),
                                                           str('%.4f' % (
                                                               recall/10)), str('%.4f' % (acc/10)), str('%.4f' % (speci/10)),
                                                           str('%.4f' % (sensi/10))))
f = open("setD.txt", "a+")
f.write("{:<7} | {:<7} {:<7} {:<7} {:<7} {:<7} {:<7}".format(
    'Method', 'F1score', 'Precisi', 'Recall', 'Accurac', 'Specifi', 'Sensiti'))
f.write("=========================================================================\n")
f.write("{:<7} | {:<7} {:<7} {:<7} {:<7} {:<7} {:<7}".format(str("BERT")[:7], str('%.4f' % (f1/10)), str('%.4f' % (preci/10)),
                                                             str('%.4f' % (
                                                                 recall/10)), str('%.4f' % (acc/10)), str('%.4f' % (speci/10)),
                                                             str('%.4f' % (sensi/10))))
f.close()